# Interacting with External Data Sources
In this notebook, we will focus on how Spark SQL interfaces with external components. Specifically, we discuss how Spark SQL allows you to:
- Use user-defined functions for both Apache Hive and Apache Spark.
- Connect with external data sources such as JDBC and SQL databases, PostgreSQL, MySQL, Tableau, Azure Cosmos DB, and MS SQL Server.
- Work with simple and complex types, higher-order functions, and common relationa operators.

## Spark SQL and Apache Hive
Spark SQL lets Spark programmers leverage the benefits of faster performance and relational programming (e.g., declarative queries and optimized storage), as well as call complex analytics libraries (e.g., machine learning).

### User-Defined Functions
Apache Spark provides the flexibility that allows for data engineers and data scientists to define their own functions too. These are known as `user-defined functions (UDFs)`.

#### Spark SQL UDFs
The benefit of creating your own PySpark UDFs is that you (and others) will be able to make use of them within Spark SQL itself. For example, a data scientist can wrap an ML model within a UDF so that a data analyst can query its predictions in Spark SQL without necessarily understanding the internals of the model.

In [3]:
import findspark

# If you know spark path you can specify it as init function parameter
findspark.init()

In [4]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = (SparkSession
            .builder
            .appName("SparkSQLandDFsPart2")
            .getOrCreate())

In [ ]:
from pyspark.sql.types import LongType

# Create cubed function
def cubed(s):
    return s * s * s

# Register UDF
spark.udf.register("cubed", cubed, LongType())

# Generate temporary view
spark.range(1, 9).createOrReplaceTempView("udf_test")

# Query the cubed UDF
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test").show()

#### Evaluation order and null checking in Spark SQL
Spark SQL (this includes SQL, the DataFrame API, and the Dataset API) does not guarantee the order of evaluation of subexpressions. For example, the following query does not guarantee that the s is NOT NULL clause is executed prior to the strlen(s)

In [ ]:
spark.sql("SELECT s FROM test1 WHERE s IS NOT NULL AND strlen(s) > 1")

Therefore, to perform proper `null` checking, it is recommended that you do the following:
1. Make the UDF itself null-aware and do n`ull checking inside the UDF.
2. Use `IF` or `CASE WHEN` expressions to do the `null` check and invoke the UDF in a conditional branch.

#### Speeding up and distributing PySpark UDFs with Pandas UDFs
One of the previous prevailing issues with using PySpark UDFs was that they had slower performance than Scala UDFs. This was because the PySpark UDFs required data movement between the JVM and Python, which was quite expensive. To resolve this problem, Pandas UDFs (also known as vectorized UDFs). A Pandas UDF uses Apache Arrow to transfer data and Pandas to work with the data. You define a Pandas UDF using the keyword pandas_udf as the decorator. Instead of operating on individual inputs row by row, you are operating on a Pandas Series or DataFrame (i.e., vectorized execution).

In [ ]:
# Import pandas
import pandas as pd

# Import various pyspark SQL functions including pandas_udf
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the cubed function
def cubed(a: pd.Series) -> pd.Series:
    return a * a * a

# Create the pandas UDF for the cubed function
cubed_udf = pandas_udf(cubed, returnType=LongType())

Let’s start with a simple Pandas Series (as defined for x) and then apply the local function `cubed()` for the cubed calculation:

In [ ]:
# Create a Pandas Series
x = pd.Series([1, 2, 3])

# The function for a pandas_udf executed with local Pandas data
print(cubed(x))

Now let’s switch to a Spark DataFrame. We can execute this function as a Spark vectorized UDF as follows:

In [ ]:
# Create a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.range(1, 4)

# Execute function as a Spark vectorized UDF
df.select("id", cubed_udf(col("id"))).show()

## Querying with the Spark SQL Shell, Beeline
There are various mechanisms to query Apache Spark, including the Spark SQL shell, the Beeline CLI utility, and reporting tools like Tableau and <a href="https://learn.microsoft.com/en-gb/azure/databricks/partners/bi/power-bi">Power BI</a>.
### Using the Spark SQL Shell
A convenient tool for executing Spark SQL queries is the spark-sql CLI. While this utility communicates with the Hive metastore service in local mode, it does not talk to the `Thrift JDBC/ODBC server` (a.k.a. Spark Thrift Server or STS). `The STS allows JDBC/ODBC clients to execute SQL queries over JDBC and ODBC protocols on Apache Spark`.
To start the Spark SQL CLI, execute the following command in the `$SPARK_HOME` folder:

In [ ]:
./bin/spark-sql

#### Create a table

In [ ]:
CREATE TABLE people (name STRING, age int);

#### Insert data into the table

In [ ]:
-- Insert from another existing table
INSERT INTO people SELECT name, age FROM ...

-- Insert values directly
INSERT INTO people VALUES ("Michael", NULL);

#### Running a Spark SQL query

In [ ]:
SHOW TABLES;

SELECT * FROM people WHERE age < 20;

SELECT name FROM people WHERE age IS NULL;

### Working with Beeline
If you’ve worked with Apache Hive you may be familiar with the command-line tool Beeline, a common utility for running HiveQL queries against HiveServer2. Beeline is a JDBC client based on the SQLLine CLI. You can use this same utility to execute Spark SQL queries against the Spark Thrift server

<b>Note!</b> Spark thrift server is pretty similar to hiveserver2 thrift, rather submitting the sql queries as hive mr job it will use spark SQL engine which underline uses full spark capabilities. As an use case tools like Tableau can easily connect to spark thrift server through ODBC driver just like hiveserver2 and access the hive or spark temp tables to run the sql queries on spark framework.

#### Connect to the Thrift server via Beeline
To test the Thrift JDBC/ODBC server using Beeline, execute the following command:

In [ ]:
./bin/beeline

In [ ]:
!connect jdbc:hive2://<hostname>:<port>

In [ ]:
SHOW tables;

SELECT * FROM people;

## External Data Sources

### JDBC and SQL Databases
Spark SQL includes a data source API that can read data from other databases using JDBC. It simplifies querying these data sources as it returns the results as a Data‐Frame.
To get started, you will need to specify the JDBC driver for your JDBC data source and it will need to be on the Spark classpath. From the `$SPARK_HOME` folder, you’ll issue a command like the following:

<b>Note!</b> JDBC common connection properties in this <a href="https://spark.apache.org/docs/latest/sql-data-sources-jdbc.html">link</a>.

#### The importance of partitioning
When transferring large amounts of data between Spark SQL and a JDBC external source, it is important to partition your data source. All of your data is going through one driver connection, which can saturate and significantly slow down the performance of your extraction, as well as  potentially saturate the resources of your source system. While these JDBC properties are optional, for any large-scale operations it is highly recommended to use the properties shown below:
| Property name      | Description |
| :----:      |    :----   |
| numPartitions      | The maximum number of partitions that can be used for parallelism in table reading and writing. This also determines the maximum number of concurrent JDBC connections.       |
| partitionColumn   | When reading an external source, `partitionColumn` is the column that is used to determine the partitions; note, `partitionColumn` must be a numeric, date, or timestamp column.        |
| lowerBound   | Sets the minimum value of `partitionColumn` for the partition stride.        |
| upperBound   | Sets the maximum value of `partitionColumn` for the partition stride.        |

Example:
- numPartitions: 10
- lowerBound: 1000
- upperBound: 10000

Then the stride is equal to 1,000, and 10 partitions will be created. This is the equivalent of executing these 10 queries (one for each partition):
- SELECT * FROM table WHERE partitionColumn BETWEEN 1000 and 2000
- SELECT * FROM table WHERE partitionColumn BETWEEN 2000 and 3000
- ...
- SELECT * FROM table WHERE partitionColumn BETWEEN 9000 and 10000

While not all-encompassing, the following are some hints to keep in mind when using these properties:
- A good starting point for numPartitions is to use a multiple of the number of Spark workers. For example, if you have four Spark worker nodes, then perhaps start with 4 or 8 partitions. But it is also important to note how well your source system can handle the read requests.
- Initially, calculate the lowerBound and upperBound based on the minimum and maximum partitionColumn actual values. For example, if you choose
{numPartitions:10, lowerBound: 1000, upperBound: 10000}, but all of the values are between 2000 and 5000, then only 3 of the 10 queries (one for each partition) will be doing all of the work. In this scenario, a better configuration would be {numPartitions:10, lowerBound: 2000, upperBound: 4000}.
- Choose a partitionColumn that can be uniformly distributed to avoid data skew. For example, if the majority of your partitionColumn has the value 2500, with {numPartitions:10, lowerBound: 1000, upperBound: 10000} most of the work will be performed by the task requesting the values between 2000 and 3000. Instead, choose a different partitionColumn, or if possible generate a new one (perhaps a hash of multiple columns) to more evenly distribute your partitions.

### PostgreSQL
To connect to a PostgreSQL database, build or download the JDBC jar from Maven and add it to your classpath. Then start a Spark shell (spark-shell or pyspark), specifying that jar:

In [ ]:
bin/spark-shell --jars postgresql-42.2.6.jar

The following examples show how to load from and save to a PostgreSQL database using the Spark SQL data source API and JDBC in Python:

In [ ]:
# Read Option 1: Loading data from a JDBC source using load method
jdbcDF1 = (spark
            .read
            .format("jdbc")
            .option("url", "jdbc:postgresql://[DBSERVER]")
            .option("dbtable", "[SCHEMA].[TABLENAME]")
            .option("user", "[USERNAME]")
            .option("password", "[PASSWORD]")
            .load())

# Read Option 2: Loading data from a JDBC source using jdbc method
# jdbcDF2 = (spark
#             .read
#             .jdbc("jdbc:postgresql://[DBSERVER]", "[SCHEMA].[TABLENAME]", properties={"user": "[USERNAME]", "password": "[PASSWORD]"}))

In [ ]:
# Write Option 1: Saving data to a JDBC source using save method
(jdbcDF1
    .write
    .format("jdbc")
    .option("url", "jdbc:postgresql://[DBSERVER]")
    .option("dbtable", "[SCHEMA].[TABLENAME]")
    .option("user", "[USERNAME]")
    .option("password", "[PASSWORD]")
    .save())

# Write Option 2: Saving data to a JDBC source using jdbc method
# (jdbcDF2
#     .write
#     .jdbc("jdbc:postgresql:[DBSERVER]", "[SCHEMA].[TABLENAME]",
#     properties={"user": "[USERNAME]", "password": "[PASSWORD]"}))

### MySQL
The following examples show how to load data from and save it to a MySQL database using the Spark SQL data source API and JDBC in Scala:

In [ ]:
# Loading data from a JDBC source using load
jdbcDF = (spark
            .read
            .format("jdbc")
            .option("url", "jdbc:mysql://[DBSERVER]:3306/[DATABASE]")
            .option("driver", "com.mysql.jdbc.Driver")
            .option("dbtable", "[TABLENAME]")
            .option("user", "[USERNAME]")
            .option("password", "[PASSWORD]")
            .load())

# Saving data to a JDBC source using save
(jdbcDF
    .write
    .format("jdbc")
    .option("url", "jdbc:mysql://[DBSERVER]:3306/[DATABASE]")
    .option("driver", "com.mysql.jdbc.Driver")
    .option("dbtable", "[TABLENAME]")
    .option("user", "[USERNAME]")
    .option("password", "[PASSWORD]")
    .save())

## Built-in Functions for Complex Data Types
Instead of using these potentially expensive techniques, you may be able to use some of the built-in functions for complex data types included as part of Apache Spark 2.4 and later. Some of the more common ones are listed in this <a href="https://sparkbyexamples.com/spark/spark-sql-array-functions/">link</a>.

### Higher-Order Funtions
In addition to the previously noted built-in functions, there are higher-order functions that take anonymous lambda functions as arguments. Let’s create a sample data set so we can run some examples:

In [ ]:
from pyspark.sql.types import *

schema = StructType([StructField("celsius", ArrayType(IntegerType()))])

t_list = [[35, 36, 32, 30, 40, 42, 38]], [[31, 32, 34, 55, 56]]

t_c = spark.createDataFrame(t_list, schema)

t_c.createOrReplaceTempView("tC")

#### transform()
The transform() function produces an array by applying a function to each element of the input array (similar to a map() function):

In [ ]:
spark.sql("""SELECT celsius,
             transform(celsius, t -> ((t * 9) div 5) + 32) as fahrenheit
             FROM tC""").show()

#### filter()
The filter() function produces an array consisting of only the elements of the input array for which the Boolean function is true:

In [ ]:
# Filter temperatures > 38C for array of temperatures
spark.sql("""SELECT celsius,
             filter(celsius, t -> t > 38) as high
             FROM tC""").show()

#### exists()
The exists() function returns true if the Boolean function holds for any element in the input array:

In [ ]:
# Is there a temperature of 38C in the array of temperatures
spark.sql("""SELECT celsius,
             exists(celsius, t -> t = 38) as threshold
             FROM tC""").show()

#### reduce()
The reduce() function reduces the elements of the array to a single value by merging the elements into a buffer B using function<B, T, B> and applying a finishing function<B, R> on the final buffer:

In [ ]:
# Calculate average temperature and convert to F
spark.sql("""SELECT celsius,
            reduce(celsius,
            0,
            (t, acc) -> t + acc,
            acc -> (acc div size(celsius) * 9 div 5) + 32
            ) as avgFahrenheit
            FROM tC""").show()

## Common DataFrames and Spark SQL Operations
Part of the power of Spark SQL comes from the wide range of DataFrame operations (also known as untyped Dataset operations) it supports. The list of operations is quite extensive and includes:
- Aggregate functions
- Collection functions
- Datetime functions
- Math functions
- Miscellaneous functions
- Non-aggregate functions
- Sorting functions
- String functions
For the full lit check the following <a href="https://spark.apache.org/docs/latest/api/sql/index.html">link</a>.

In [5]:
file_location = "../data/movie_data_part1.csv"

df_mani = spark.read.format("csv") \
            .option("inferSchema", True) \
            .option("header", True) \
            .option("sep", '|') \
            .load(file_location)
            
df_mani.count()

43998

### Subset Columns and View a Glimpse of the Data

In [6]:
# Defining a list to subset the required columns
select_columns=['id','budget','popularity','release_date','revenue','title']

# Subsetting the required columns from the DataFrame
df_mani = df_mani.select(*select_columns)

# The following command displays the data; by default it shows top 20 rows
df_mani.show()

+-----+---------+------------------+------------+---------+--------------------+
|   id|   budget|        popularity|release_date|  revenue|               title|
+-----+---------+------------------+------------+---------+--------------------+
|43000|      0.0|             2.503|  1962-05-23|      0.0|The Elusive Corporal|
|43001|      0.0|              5.51|  1962-11-12|      0.0|  Sundays and Cybele|
|43002|      0.0|              5.62|  1962-05-24|      0.0|Lonely Are the Brave|
|43003|      0.0|             7.159|  1975-03-12|      0.0|          F for Fake|
|43004| 500000.0|             3.988|  1962-10-09|      0.0|Long Day's Journe...|
|43006|      0.0|             3.194|  1962-03-09|      0.0|           My Geisha|
|43007|      0.0|             2.689|  1962-10-31|      0.0|Period of Adjustment|
|43008|      0.0|             6.537|  1959-03-13|      0.0|    The Hanging Tree|
|43010|      0.0|             4.297|  1962-01-01|      0.0|Sherlock Holmes a...|
|43011|      0.0|           

### Missing Values

In [14]:
from pyspark.sql.functions import *

df_mani.filter((df_mani['popularity']=='') | col('popularity').isNull() | isnan('popularity')).count()

215

If you need to calculate all the missing values in the DataFrame, you can use thefollowing command:

In [18]:
df_mani.select([count(when((col(c)=='') | col(c).isNull() | isnan(c), lit('null'))).alias(c) for c in df_mani.columns]).show()

+---+------+----------+------------+-------+-----+
| id|budget|popularity|release_date|revenue|title|
+---+------+----------+------------+-------+-----+
|125|   125|       215|         221|    215|  304|
+---+------+----------+------------+-------+-----+



### One-Way Frequencies
Let’s see how we can calculate the frequencies of categorical variables:

In [27]:
df_mani.groupBy(col('title')).count().sort(desc("count")).show(10, False)

+--------------------+-----+
|title               |count|
+--------------------+-----+
|null                |304  |
|The Three Musketeers|8    |
|Les Misérables      |8    |
|Cinderella          |8    |
|Hamlet              |7    |
|Frankenstein        |7    |
|A Christmas Carol   |7    |
|The Island          |7    |
|Dracula             |7    |
|Framed              |6    |
+--------------------+-----+
only showing top 10 rows



In [26]:
df_temp = df_mani.filter((df_mani['title']!='') & (df_mani['title'].isNotNull()) & (~isnan(df_mani['title'])))

df_temp.show(10, False)

+-----+--------+----------+------------+-------+---------------------------------------+
|id   |budget  |popularity|release_date|revenue|title                                  |
+-----+--------+----------+------------+-------+---------------------------------------+
|43000|0.0     |2.503     |1962-05-23  |0.0    |The Elusive Corporal                   |
|43001|0.0     |5.51      |1962-11-12  |0.0    |Sundays and Cybele                     |
|43002|0.0     |5.62      |1962-05-24  |0.0    |Lonely Are the Brave                   |
|43003|0.0     |7.159     |1975-03-12  |0.0    |F for Fake                             |
|43004|500000.0|3.988     |1962-10-09  |0.0    |Long Day's Journey Into Night          |
|43006|0.0     |3.194     |1962-03-09  |0.0    |My Geisha                              |
|43007|0.0     |2.689     |1962-10-31  |0.0    |Period of Adjustment                   |
|43008|0.0     |6.537     |1959-03-13  |0.0    |The Hanging Tree                       |
|43010|0.0     |4.297

In [25]:
df_temp.groupby(df_temp['title']).count().filter("`count` > 4").sort(col("count").desc()).show(10, False)

+--------------------+-----+
|title               |count|
+--------------------+-----+
|Cinderella          |8    |
|The Three Musketeers|8    |
|Les Misérables      |8    |
|Hamlet              |7    |
|Frankenstein        |7    |
|The Island          |7    |
|Dracula             |7    |
|A Christmas Carol   |7    |
|First Love          |6    |
|Beauty and the Beast|6    |
+--------------------+-----+
only showing top 10 rows



### Casting Variables
If you are not careful in identifying the datatypes, you may experience data loss on casting. For example, if you are converting a string column to a numeric one, the resulting column can be all nulls. It is good practice to explore the dataset before applying any transformations.

In [28]:
df_mani.dtypes

[('id', 'string'),
 ('budget', 'double'),
 ('popularity', 'double'),
 ('release_date', 'string'),
 ('revenue', 'double'),
 ('title', 'string')]

In [29]:
df_mani = df_mani.withColumn('budget', df_temp['budget'].cast("float"))

In [30]:
df_mani.dtypes

[('id', 'string'),
 ('budget', 'float'),
 ('popularity', 'double'),
 ('release_date', 'string'),
 ('revenue', 'double'),
 ('title', 'string')]

In [37]:
#Importing necessary libraries
from pyspark.sql.types import *

#Identifying and assigning lists of variables
int_vars=['id']
float_vars=['budget', 'popularity', 'revenue']
date_vars=['release_date']

#Converting integer variables
for column in int_vars:
    df_mani = df_temp.withColumn(column, df_temp[column].cast(IntegerType()))

for column in float_vars:
    df_mani = df_temp.withColumn(column, df_temp[column].cast(FloatType()))

for column in date_vars:
    df_mani = df_temp.withColumn(column, df_temp[column].cast(DateType()))

df_mani.dtypes

[('id', 'int'),
 ('budget', 'float'),
 ('popularity', 'float'),
 ('release_date', 'date'),
 ('revenue', 'float'),
 ('title', 'string')]

### Descriptive Statistics
To analyze any data, you should have a keen understanding of the type of data, its distribution, and its dispersion. Spark has a nice suite of built-in functions that will make it easier to quickly calculate these fields. The describe function in Spark is very handy, as it gives the count of total non-missing values for each column, mean/average, standard deviation, and minimum and maximum values.

In [36]:
df_mani.describe().show()

+-------+--------------------+--------------------+------------------+--------------------+-------------------+--------------------+
|summary|                  id|              budget|        popularity|        release_date|            revenue|               title|
+-------+--------------------+--------------------+------------------+--------------------+-------------------+--------------------+
|  count|               43873|               43873|             43783|               43777|              43783|               43694|
|   mean|  44502.304312077475|  3736901.8351683044| 5.295444259187363|                null|  9697079.597131306|            Infinity|
| stddev|  27189.646588626394|1.5871814953840714E7| 6.168030519876136|                null|5.687938447592969E7|                 NaN|
|    min|"[{'id': 104, 'lo...|                 0.0|0.6000000000000001|        1 Giant Leap|                0.0|!Women Art Revolu...|
|    max|[{'id': 9327, 'lo...|               3.8E8|             180.0

### Unique/Distinct Values and Counts
You may sometimes just want to know the number of levels (cardinality) within a variable. You can do this using the countDistinct function available in Spark

In [35]:
# Counts the distinct occurances of titles
df_mani.agg(countDistinct(col("title")).alias("count")).show()

+-----+
|count|
+-----+
|41138|
+-----+



In [39]:
# Counts the distinct occurances of titles
df_mani.select('title').distinct().show(10,False)

+---------------------------------------------+
|title                                        |
+---------------------------------------------+
|The Corn Is Green                            |
|Meet The Browns - The Play                   |
|Morenita, El Escandalo                       |
|Father Takes a Wife                          |
|The Werewolf of Washington                   |
|My Wife Is a Gangster                        |
|Depeche Mode: Touring the Angel Live in Milan|
|A Woman Is a Woman                           |
|History Is Made at Night                     |
|Colombian Love                               |
+---------------------------------------------+
only showing top 10 rows



In [42]:
# Extracting year from the release date
df_temp = df_mani.withColumn('release_year', year('release_date'))

# Extracting month
df_temp=df_temp.withColumn('release_month', month('release_date'))

# Extracting day of month
df_temp=df_temp.withColumn('release_day', dayofmonth('release_date'))

# Calculating the distinct counts by the year
df_temp.groupBy("release_year").agg(countDistinct("title")).show(10,False)

+------------+------------+
|release_year|count(title)|
+------------+------------+
|1959        |271         |
|1990        |496         |
|1975        |365         |
|1977        |415         |
|1924        |19          |
|2003        |1199        |
|2007        |1896        |
|2018        |4           |
|1974        |434         |
|2015        |13          |
+------------+------------+
only showing top 10 rows



### Filtering

In [43]:
# Filter all the titles that start with “Meet”
df_mani.filter(df_mani['title'].like('Meet%')).show(10,False)

+-----+---------+----------+------------+-----------+--------------------------+
|id   |budget   |popularity|release_date|revenue    |title                     |
+-----+---------+----------+------------+-----------+--------------------------+
|43957|500000.0 |2.649     |2005-06-28  |1000000.0  |Meet The Browns - The Play|
|39997|0.0      |3.585     |1989-11-15  |0.0        |Meet the Hollowheads      |
|16710|0.0      |11.495    |2008-03-21  |4.1939392E7|Meet the Browns           |
|20430|0.0      |3.614     |2004-01-29  |0.0        |Meet Market               |
|76435|0.0      |1.775     |2011-03-31  |0.0        |Meet the In-Laws          |
|76516|5000000.0|4.05      |1990-11-08  |485772.0   |Meet the Applegates       |
|7278 |3.0E7    |11.116    |2008-01-24  |8.4646832E7|Meet the Spartans         |
|32574|0.0      |7.42      |1941-03-14  |0.0        |Meet John Doe             |
|40506|0.0      |4.814     |1997-01-31  |0.0        |Meet Wally Sparks         |
|40688|2.4E7    |6.848     |

In [44]:
# Find out the titles that do not end with an “s”
df_mani.filter(~df_mani['title'].like('%s')).show(10,False)

+-----+--------+----------+------------+-------+---------------------------------------+
|id   |budget  |popularity|release_date|revenue|title                                  |
+-----+--------+----------+------------+-------+---------------------------------------+
|43000|0.0     |2.503     |1962-05-23  |0.0    |The Elusive Corporal                   |
|43001|0.0     |5.51      |1962-11-12  |0.0    |Sundays and Cybele                     |
|43002|0.0     |5.62      |1962-05-24  |0.0    |Lonely Are the Brave                   |
|43003|0.0     |7.159     |1975-03-12  |0.0    |F for Fake                             |
|43004|500000.0|3.988     |1962-10-09  |0.0    |Long Day's Journey Into Night          |
|43006|0.0     |3.194     |1962-03-09  |0.0    |My Geisha                              |
|43007|0.0     |2.689     |1962-10-31  |0.0    |Period of Adjustment                   |
|43008|0.0     |6.537     |1959-03-13  |0.0    |The Hanging Tree                       |
|43010|0.0     |4.297

In [45]:
# Find any title that contains “ove,” we could use the rlike function, which is a regular expression
df_mani.filter(df_mani['title'].rlike('\w*ove')).show(10,False)
# df_mani.filter(df_mani.title.contains('ove')).show()

+-----+------+----------+------------+------------+------------------------+
|id   |budget|popularity|release_date|revenue     |title                   |
+-----+------+----------+------------+------------+------------------------+
|43100|0.0   |7.252     |1959-10-07  |0.0         |General Della Rovere    |
|43152|0.0   |5.126     |2001-06-21  |0.0         |Love on a Diet          |
|43191|0.0   |4.921     |1952-08-29  |0.0         |Beware, My Lovely       |
|43281|0.0   |2.411     |1989-11-22  |0.0         |Love Without Pity       |
|43343|0.0   |3.174     |1953-12-25  |0.0         |Easy to Love            |
|43347|3.0E7 |14.863    |2010-11-22  |1.02820008E8|Love & Other Drugs      |
|43362|0.0   |1.705     |1952-02-23  |0.0         |Love Is Better Than Ever|
|43363|0.0   |2.02      |1952-05-29  |0.0         |Lovely to Look At       |
|43395|0.0   |4.758     |1950-11-10  |0.0         |Two Weeks with Love     |
|43455|0.0   |4.669     |1948-08-23  |0.0         |The Loves of Carmen     |

### Deleting and Renaming Columns
You can always drop any column or columns using the drop function.

In [ ]:
columns_to_drop=['budget_cat']

# df_with_newcols = df_with_newcols.drop(*columns_to_drop)
# df_with_newcols = df_with_newcols.withColumnRenamed('id','film_id') .withColumnRenamed('ratings','film_ratings')

If you would like to change multiple column names, you try the following command:

In [51]:
new_names = [('budget','film_budget'),('popularity','film_popularity')]

print(*zip(*new_names))

('budget', 'film_budget') ('popularity', 'film_popularity')


In [ ]:
# Applying the alias function
df_with_newcols_renamed = df_with_newcols.select(list(map(lambda old,new: col(old).alias(new), *zip(*new_names))))